In [1]:
from src.reachability_analysis.simulation import get_test_config, get_test_label, get_cluster, get_initial_conditions, reachability_for_all_modes
from src.datasets.data import SVEAData, ROSData
from src.transformer_model.model import create_model, evaluate
import json
import logging
import os
import rospy
from torch.utils.data import DataLoader
from src.datasets.data import data_factory, Normalizer
from src.datasets.masked_datasets import collate_unsuperv
from src.utils.load_data import load_task_datasets


logging.basicConfig(
    format="%(asctime)s | %(levelname)s : %(message)s", level=logging.INFO
)
logger = logging.getLogger(__name__)

/home/sam/Desktop/Pedestrian_Project/.venv/lib/python3.9/site-packages/pypolycontain/operations.py:23: UserWarning: You don't have pydrake installed properly. Methods that rely on optimization may fail.
  warnings.warn("You don't have pydrake installed properly. Methods that rely on optimization may fail.")
/home/sam/Desktop/Pedestrian_Project/.venv/lib/python3.9/site-packages/pypolycontain/conversions.py:19: UserWarning: WARNING: You don't have CDD package installed. Unable to visualize polytopes. You may still visualize zonotopes.
  warnings.warn("WARNING: You don't have CDD package installed. Unable to visualize polytopes. You may still visualize zonotopes.")
/home/sam/Desktop/Pedestrian_Project/.venv/lib/python3.9/site-packages/pypolycontain/conversions.py:32: UserWarning: You don't have pydrake installed properly. Methods that rely on optimization may fail.
  warnings.warn("You don't have pydrake installed properly. Methods that rely on optimization may fail.")
/home/sam/Desktop/P

In [2]:
config_path = '/home/sam/Desktop/Pedestrian_Project/experiments/SINDDataset_pretrained_2024-04-27_00-11-45_KIP/configuration.json'

with open(config_path) as cnfg:
    config = json.load(cnfg)

ROOT_RESOURCES = os.getcwd() + "/resources"


config['original_data'] = False

config_test = config.copy()
config_test['online_data'] = True
config_test['online_data'] = False
config_test['pattern'] = None
config_test['data_dir'] = '/home/sam/Desktop/Pedestrian_Project/bags'
config_test['data_class'] = 'svea'
config_test['eval_only'] = True
config_test['val_ratio'] = 1

In [3]:
data_oracle = ROSData(config_test)


2024-07-08 18:02:47,622 | INFO : Loading 2 datasets files using 2 parallel processes ...


[INFO]  Data folder /home/sam/Desktop/Pedestrian_Project/bags/ismir_data_2024-06-28-11-56-17 already exists. Not creating.
[INFO]  Data folder /home/sam/Desktop/Pedestrian_Project/bags/ismir_data_2024-06-28-11-57-29 already exists. Not creating.


In [4]:
# online_Data = DataLoader(
#     dataset=data_oracle.all_df,
#     batch_size=config["batch_size"],
#     shuffle=False,
#     num_workers=config["num_workers"],
#     pin_memory=True,
#     collate_fn=lambda x: collate_fn(x, max_len=data_oracle.max_seq_len),
# )

val_data = data_oracle.feature_df

# Pre-process features
if config_test["data_normalization"] != "none":
    logger.info("Normalizing data ...")
    normalizer = Normalizer(config_test["data_normalization"])
    val_data = normalizer.normalize(val_data)


task_dataset, collate_fn = load_task_datasets(config_test)
val_dataset = task_dataset(data_oracle.feature_df, [0])

# TODO there is a problem here

# Dataloaders
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=config_test["batch_size"],
    shuffle=False,
    num_workers=config_test["num_workers"],
    pin_memory=True,
    collate_fn=lambda x: collate_fn(x, max_len=data_oracle.max_seq_len),
)

In [5]:
val_loader.dataset

In [16]:
config_test['eval_only']

True

In [6]:
model, optimizer, trainer, val_evaluator, start_epoch = create_model(
    config_test, None, val_loader, data_oracle, logger, device='cpu'
)

/home/sam/Desktop/Pedestrian_Project/.venv/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
2024-07-08 18:02:48,746 | INFO : Model:
TSTransformerEncoder(
  (project_inp): Linear(in_features=6, out_features=128, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Li

In [7]:
trainer, val_evaluator.dataloader

(<src.transformer_model.model.UnsupervisedAttentionModel at 0x7fef7c076a30>,
 <torch.utils.data.dataloader.DataLoader at 0x7fee6d1f3f10>)

In [8]:
config_test['eval_only'] = True
config_test['val_ratio'] = 1

In [9]:
aggr_metrics, embedding_data = evaluate(val_evaluator, config=config_test, save_embeddings=True)


2024-07-08 18:02:49,676 | INFO : Evaluation Summary: loss: 386.078698 | 
2024-07-08 18:02:49,677 | INFO : Evaluation runtime: 0.0 hours, 0.0 minutes, 0.010312318801879883 seconds



Evaluating   0.0% | batch:         0 of         1	|	loss: 386.079


In [10]:
embedding_data.keys()

dict_keys(['target_masks', 'targets', 'predictions', 'metrics', 'IDs', 'padding_masks', 'embeddings', 'embeddings_original'])

In [11]:
len(embedding_data['embeddings'])

1

In [14]:
embedding_data['IDs']

[(0,)]

In [12]:
embedding_data['embeddings'][0].shape

(1, 50, 128)

In [13]:
output = []
# TODO iterate through trajectories

embedding = embedding_data['embeddings'][0].shape
c = get_cluster(config, data_oracle) # TODO can you batch with AnnoyModel?
nn_model = AnnoyModel(config=config)
test_cases = {f'c_{c}': f'Cluster: {c}'}
pos, v = get_initial_conditions(trajectory)
z, l, _b, _z = reachability_for_all_modes(pos=pos, vel=v, baseline=False, test_cases=test_cases, config=config, trajectory=trajectory, show_plot=True, save_plot=None)



    # output.append(z[0])

/home/sam/Desktop/Pedestrian_Project/.venv/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
2024-07-08 18:02:49,745 | INFO : Model:
TSTransformerEncoder(
  (project_inp): Linear(in_features=6, out_features=128, bias=True)
  (pos_enc): LearnablePositionalEncoding(
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerBatchNormEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Li

Evaluating   0.0% | batch:         0 of         1	|	loss: 183.426
Load data from: /home/sam/Desktop/Pedestrian_Project/resources


FileNotFoundError: [Errno 2] No such file or directory: '/home/sam/Desktop/Pedestrian_Project/resources/data_embeddings.pkl'